# Introduction to NLP Fundamentals in Tensorflow

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

6192
Target: 0, (not a real disaster)
Text:
@phnotf sometimes your cheekiness bleeds through my computer screen and i recoil in fear

---
Target: 1, (real disaster)
Text:
Skinny Jeans are Hazardous for Your Health! #socialnews http://t.co/pAQanenCeS

---
Target: 0, (not a real disaster)
Text:
@BaseballQuotes1 I have a 32 inch dynasty

---
Target: 0, (not a real disaster)
Text:
I'm ready to explode! http://t.co/OwJe3i6yGN

---
Target: 0, (not a real disaster)
Text:
@Silverhusky Shtap!  Before your town is obliterated and the earth is salted T_T

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
People who say it cannot be done should not interrupt those who are doing it. ÛÒ George Bernard Shaw      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  57,   65,  132,   15, 1428,   21,  578,  168,   34,    1,  161,
          65,   22,  487,   15]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Dust Storm 'en route' from Alice Springs to Uluru http://t.co/4ilt6FXU45

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02825609, -0.03884822,  0.01537738, ..., -0.0465418 ,
         -0.01634668, -0.02971794],
        [ 0.01406411, -0.02755634, -0.01654502, ..., -0.0248298 ,
          0.00778814,  0.01590942],
        [ 0.00711604, -0.04026858, -0.02276772, ...,  0.0333466 ,
          0.01716986,  0.04186622],
        ...,
        [ 0.00927341,  0.02309536, -0.04119989, ...,  0.04156239,
         -0.01576883, -0.01418631],
        [ 0.00927341,  0.02309536, -0.04119989, ...,  0.04156239,
         -0.01576883, -0.01418631],
        [ 0.00927341,  0.02309536, -0.04119989, ...,  0.04156239,
         -0.01576883, -0.01418631]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 2.82560922e-02, -3.88482213e-02,  1.53773762e-02, -1.96116101e-02,
        5.04815578e-03, -4.79421392e-02,  2.29966640e-03, -2.32868548e-02,
        4.52186503e-02, -2.37231497e-02,  2.30169781e-02, -6.32773712e-03,
        2.04688795e-02, -4.54888344e-02, -2.49351189e-03, -3.70902196e-02,
        1.39306225e-02,  1.85836293e-02, -4.30788994e-02,  3.73834483e-02,
        1.52803585e-03,  1.69031955e-02,  4.41368558e-02,  1.69019736e-02,
        2.93903090e-02,  4.84101437e-02,  1.45431645e-02, -1.78015716e-02,
        2.58936174e-02,  3.93015854e-02, -3.11392304e-02, -8.16118717e-03,
       -3.06976326e-02,  4.33182716e-03, -8.63688067e-03,  3.38607170e-02,
        4.81161214e-02, -4.97042798e-02,  3.62318270e-02, -4.72672097e-02,
       -2.97130942e-02, -4.20909896e-02,  2.57658996e-02, -3.20958123e-02,
       -4.67848778e-03,  2.87621506e-02, -3.83338332e-02,  4.06917222e-02,
       -7.31257349e-03,  3.33312787e-02,  4.48244102

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230812-084222


Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6096 - accuracy: 0.6917 - val_loss: 0.5363 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4418 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3470 - accuracy: 0.8606 - val_loss: 0.4589 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2855 - accuracy: 0.8923 - val_loss: 0.4637 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2385 - accuracy: 0.9118 - val_loss: 0.4760 - val_accuracy: 0.7874


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4760 - accuracy: 0.7874


[0.4760214388370514, 0.787401556968689]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)


(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230812-084231
Epoch 1/5


215/215 [==============================] - 7s 21ms/step - loss: 0.5071 - accuracy: 0.7508 - val_loss: 0.4605 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3266 - accuracy: 0.8698 - val_loss: 0.5173 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2302 - accuracy: 0.9134 - val_loss: 0.5630 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1692 - accuracy: 0.9407 - val_loss: 0.6332 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1206 - accuracy: 0.9539 - val_loss: 0.7252 - val_accuracy: 0.7638


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.03741802],
       [0.59196067],
       [0.9977483 ],
       [0.06120349],
       [0.00876346],
       [0.9984079 ],
       [0.938054  ],
       [0.99922395],
       [0.99880326],
       [0.10394315]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.37795275590551,
 'precision': 0.7676775517210905,
 'recall': 0.7637795275590551,
 'f1': 0.7604572192804401}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.2380 - accuracy: 0.9019 - val_loss: 0.5295 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1612 - accuracy: 0.9415 - val_loss: 0.5927 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1289 - accuracy: 0.9531 - val_loss: 0.7497 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.1098 - accuracy: 0.9597 - val_loss: 0.7585 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0921 - accuracy: 0.9657 - val_loss: 0.7395 - val_accuracy: 0.7913


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[5.7781488e-02],
       [8.2867342e-01],
       [9.9956495e-01],
       [4.7781654e-02],
       [8.5337006e-04],
       [9.9225849e-01],
       [6.0804921e-01],
       [9.9975866e-01],
       [9.9944776e-01],
       [6.4987457e-01]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 79.13385826771653,
 'precision': 0.793352563602817,
 'recall': 0.7913385826771654,
 'f1': 0.7895046541197648}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 7s 22ms/step - loss: 0.5063 - accuracy: 0.7516 - val_loss: 0.4551 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3125 - accuracy: 0.8736 - val_loss: 0.5234 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2062 - accuracy: 0.9194 - val_loss: 0.5642 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1385 - accuracy: 0.9540 - val_loss: 0.6845 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0989 - accuracy: 0.9658 - val_loss: 0.7256 - val_accuracy: 0.7625


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.5093 - accuracy: 0.7469 - val_loss: 0.4574 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3133 - accuracy: 0.8720 - val_loss: 0.5210 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2125 - accuracy: 0.9193 - val_loss: 0.5727 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1485 - accuracy: 0.9499 - val_loss: 0.6432 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1055 - accuracy: 0.9648 - val_loss: 0.6829 - val_accuracy: 0.7638


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.04186536],
       [0.878412  ],
       [0.99925035],
       [0.11318437],
       [0.00561359],
       [0.9954922 ],
       [0.8864701 ],
       [0.99954385],
       [0.9995747 ],
       [0.21342382]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.76369847151459,
 'recall': 0.7637795275590551,
 'f1': 0.7626922180365151}

#### 1D CNN

In [47]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [60]:
# trying on my own
# text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
#                                                     output_sequence_length=15)
# text_vectorizer.adapt(train_sentences)

# inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
# x = text_vectorizer(inputs)
# x = tf.keras.layers.Embedding(input_dim=10000,
#                               output_dim=128,
#                               input_length=15)(x)
# x = tf.keras.layers.Conv1D(64,3)(x)
# x = tf.keras.layers.GlobalMaxPool1D()(x)
# x = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

# model_5_exp = tf.keras.Model(inputs,outputs)
# model_5_exp.summary()

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.string, name='input_6'), name='input_6', description="created by layer 'input_6'") at layer "text_vectorization_4". The following previous layers were accessed without issue: []